In [2]:
!pip3 install pinecone-client openai
!pip3 install sentence-transformers
!pip3 install python-dotenv



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os  
import dotenv                                                                                                                                                                                                        
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
else:
    print("No .env file found. Please check its location.")

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY_Valor")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENV_Valor")
OPENAI_API_KEY = os.getenv("OPEN_AI_API_KEY_Valor")
access_token = os.getenv("DistilBert_AccessToken_Valor")

In [38]:
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import pinecone 
from pinecone import Pinecone

#openai.api_key = OPENAI_API_KEY


pc = Pinecone(api_key=PINECONE_API_KEY)

# Define the Pinecone index name and specs
#index_name = "subreddit-data-index-updated"
index_name = "json"
existing_indexes = pc.list_indexes().names()
print("Existing indexes:", existing_indexes)
index = Pinecone.Index(self=pc,name=index_name)


Existing indexes: ['only-comments', 'json', 'quickstart', 'subreddit-data-index', 'subreddit-data-index-updated']


In [12]:
#embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Model for generating query embeddings
embedding_model = SentenceTransformer('distilbert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct",access_token=access_token)
generation_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

No sentence-transformers model found with name distilbert-base-uncased. Creating a new one with mean pooling.


In [9]:
def generate_query_embedding(query):
    return embedding_model.encode(query).tolist()

def retrieve_documents(query, top_k=5):
    query_embedding = generate_query_embedding(query)  # Generate embedding for query
    search_results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)  # Perform vector search

    # Extract text and metadata from results
    documents = []
    for match in search_results["matches"]:
        metadata = match["metadata"]
        print(metadata)
        # Dynamically get the main content from metadata based on available keys
        body = metadata.get("text") or "No text available"
        documents.append({"body": body, "metadata": metadata})
    return documents


In [42]:
#Generate response using the context
def generate_response(context, query):
    prefix = "The context below are posts and comments from subreddit r/VAbenefits, use the relevant similar contexts and frame the answer to the question appropriately but do not mention about the context in the response. If the context is not enough, please use your own dataset to answer the question. Do not give the exact context as the answer"
    input_text = f"{prefix}\n\nContext: {context}\n\nQuestion: {query}\n\nAnswer:"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    outputs = generation_model.generate(inputs, max_length=2500, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# def generate_response(context, query):
#     input_text = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"


In [16]:
#Generate response without the context for comparison
def generate_response_without_context( query):
    input_text = f"Question: {query}\n\nAnswer:"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    outputs = generation_model.generate(inputs, max_length=1500, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [46]:
# Main pipeline
def main():
    # Example query
    #query = "Can a veteran lose their benefits if they have a drug addiction?"
    query = "should i be concerned about losing va disability if the latest assesment says that i do not have sleep apnea?"

    # Retrieve relevant documents from Pinecone
    retrieved_docs = retrieve_documents(query, top_k=3)

    # Combine retrieved documents into a single context
    context = "\n\n".join([f"Document {i+1}: {doc['body']} {doc['metadata']['comments'][:3]}" for i, doc in enumerate(retrieved_docs)])

    # Generate response
    response = generate_response(context, query)
    print("Retrieved Context:\n", context)
    print("\nGenerated Response:\n", response)


if __name__ == "__main__":
    main()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'comments': ['MH is linked to sleep apnea... Yet being a black male is linked to type 2 diabetes.\\n\\n\\nDoes being black cause diabetes?\\n\\n\\nMH does not cause sleep apnea.\\n\\n\\nIf you want to be successful with SA then you need to fine the cause, and linked that to your MH.\\n\\n\\nNexus is a "link\', but when medical journals say MH is a link, it\'s not a nexus. It might be evidence, but not a nexus.', '[removed]', 'Unless your sleep apnea was caused in service, I would recommend a nexus because is very hard to prove without being sc Secondary conditions are the key to getting it connected & a Nexus letter explains on how it is service connected to certain disabilities that your already service connected for', "Sadly, the VA doesn't care what your opinion is on this.  You need a medical professional to look at your circumstances and provide a medical opinion as to what the cause of the OSA is.  I wish it was as easy as just giving a personal opinion....100% would be the norm

In [47]:
# Main pipeline
def main():
    # Example query
    #query = "Can a veteran lose their benefits if they have a drug addiction?"
    query = "should i be concerned about losing va disability if the latest assesment says that i do not have sleep apnea?"

    # Retrieve relevant documents from Pinecone
    retrieved_docs = retrieve_documents(query, top_k=5)

    # Combine retrieved documents into a single context
    context = "\n\n".join([f"Document {i+1}: {doc['body']}" for i, doc in enumerate(retrieved_docs)])

    # Generate response
    response = generate_response_without_context(query)
    #print("Retrieved Context:\n", context)
    print("\nGenerated Response:\n", response)


if __name__ == "__main__":
    main()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'comments': ['MH is linked to sleep apnea... Yet being a black male is linked to type 2 diabetes.\\n\\n\\nDoes being black cause diabetes?\\n\\n\\nMH does not cause sleep apnea.\\n\\n\\nIf you want to be successful with SA then you need to fine the cause, and linked that to your MH.\\n\\n\\nNexus is a "link\', but when medical journals say MH is a link, it\'s not a nexus. It might be evidence, but not a nexus.', '[removed]', 'Unless your sleep apnea was caused in service, I would recommend a nexus because is very hard to prove without being sc Secondary conditions are the key to getting it connected & a Nexus letter explains on how it is service connected to certain disabilities that your already service connected for', "Sadly, the VA doesn't care what your opinion is on this.  You need a medical professional to look at your circumstances and provide a medical opinion as to what the cause of the OSA is.  I wish it was as easy as just giving a personal opinion....100% would be the norm